# Tâche 2 : Question-réponse avec modèle pré‑entraîné GPT‑2, sans adaptation

**Objectifs**

Observer les limites d’un modèle de langage **pré‑entraîné** (**GPT-2 Medium (355M)**) lorsqu’on lui pose des questions sur un sujet **absent** de ses données d’origine (ici, l’univers de *Sherlock Holmes*).
Dans ce *notebook*, vous faites **uniquement de l’inférence** avec un modèle qui a déjà été pré-entraîné. Cette tâche consiste à construire un *prompt* minimal, générer des réponses avec le modèle sans modification, puis évaluer la pertinence des résultats. Aucun entraînement de modèle n'est effectué pour cette tâche.

**Objectifs d’apprentissage**
1. Charger un modèle pré‑entraîné et son tokenizer de Hugging Face.
2. Générer du texte et **isoler la réponse** du modèle.
3. Comprendre et expliquer les **limites du pré‑entraînement** hors‑domaine.

Les **questions** pour évaluer le modèle vous sont fournies. Vous devez comprendre le format des questions chargées en mémoire.

> Il est recommandé de faire ce travail pratique en utilisant une carte graphique GPU compatible avec HuggingFace/Pytorch.
> Si votre machine n’en possède pas, vous pouvez utiliser **Google Colab** pour exécuter le *notebook* dans le cloud.

Si nécessaire, installer les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [ ]:
#!pip install datasets
#!pip install accelerate
#!pip install transformers[torch]
#!pip3 install torch torchvision

In [1]:
batch_size = 5 # il est possible d'ajuster la taille de batch. Les valeurs actuelles utilisent environ 10 Gb
max_length = 256 # on réduit le contexte pour sauver du temps, nos exemples ne nécesside pas une plus grande fenêtre de mots
model_name = "gpt2-medium"

In [ ]:
from transformers import pipeline
import os
import json

## 1. Chargement du modèle Hugging Face et du tokenizer (à compléter)

Complétez le corps de la fonction `load_model(model_path)` afin qu’elle :

- charge le **tokenizer** et le **modèle** Hugging Face à partir du chemin `model_path`.
- **retourne** le tokenizer comme **première valeur de retour** et le modèle comme **seconde valeur de retour**.

On ajoute également des fonctions pour monter les questions en mémoire et pour sauvegarder les réponses dans un fichier. 

In [ ]:
def load_model(model_path):
    tokenizer = None # TODO
    model = None # TODO
    
    return tokenizer, model

In [ ]:
def load_entries(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    if not isinstance(data, list):
        raise ValueError(f"Question file must contain a list of objects. Got: {type(data)}")
    return data

def save_answers(questions_answers, output_dir, out_file_name, display=True):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, out_file_name), "w", encoding="utf-8") as out:
        for index, question, answer, expected_answer in questions_answers:
            out.write(f"Q: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}\n")
            if display:
                print(f"Q{index}: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}")


## 2. Fonctions de test question-réponse  (à compléter)

La fonction **test_on_questions** est utilisée pour parcourir **toutes les entrées** du fichier de questions afin de produire des réponses générées par le modèle.

La génération d'une réponse à une question implique les étapes suivantes (fonction **process_entry** à compléter) : 
* Construire un prompt à l’aide de la fonction **build_prompt** (rendu disponible).
* Utiliser le modèle (via un pipeline de génération passé en argument) pour générer une réponse à une question.
* Retourner la réponse générée par le modèle.  

Points importants à souligner: 
* La fonction ***process_entry*** doit retourner uniquement la réponse générée par le modèle (sans la question - le prompt).
* Il est de votre responsabililté de choisir **les paramètres** du générateur (max_new_tokens, do_sample, temperature, top_k ou top_p). Décrivez ceux que vous avez retenus.

> Afin de simplifier le travail, nous avons choisi de ne pas utiliser de *batchs* dans la fonction qui teste les questions.
> Vous n'avez pas à prendre en compte le *warning* qui suggère d'utiliser des *datasets*.

In [ ]:
def build_prompt(entry):
    return entry.get("question", "")

Description des paramètres de génération: 
(à compléter...)

In [ ]:
def process_entry(entry, prompt_builder, generator):
    answer = None # TODO
    return answer

In [ ]:
def test_on_questions(prompt_builder, model_path, question_file, out_file_name, output_dir="results"):
    entries = load_entries(question_file)
    tokenizer, model = load_model(model_path)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    results = []
    for i, entry in enumerate(entries):
        answer = process_entry(entry, prompt_builder, generator)
        question = entry.get("question", "")
        expected_answer = entry.get("answer", "")
        results.append((i, question, answer, expected_answer))
    save_answers(results, output_dir, out_file_name, display=True)
    return results

## 3. Génération avec GPT-2 de réponses aux questions sur Sherlock Holmes

Exécutez la cellule suivante pour générer les réponses aux questions.
Le temps d’exécution devrait se situer entre **5 et 10 minutes** si vous utilisez **Google Colab** avec un GPU.

Note : N'oubliez pas d'ajouter le fichier de réponses générées par le modèle (voir *out_file_name*) dans votre remise du travail. Ainsi le fichier ZIP que vous déposerez sur le site du cours devra contenir tous les *notebooks* et tous les fichiers de réponses.

In [ ]:
questions = "data/questions_sherlock.json"
out_file_name = "gpt2_answers.txt"

results = test_on_questions(prompt_builder=build_prompt, model_path=model_name, question_file=questions, out_file_name=out_file_name)

## 4. Analyse des résultats 

### 4.1 Évaluation quantitative (à compléter)

In [ ]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0, 0.0, 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [ ]:
def evaluation_generation(results): 
    # TODO
    return eval

In [ ]:
eval = evaluation_generation(results)
print(eval)

**Question:** Que pensez-vous de cette évaluation ? 

### 4. Analyse qualitative (à faire)

Rédigez **5 à 8 phrases** expliquant ce que vous observez et pourquoi, selon vous, le modèle fournit ce type de réponses.

> Cette étape prépare le terrain pour les tâches 3 et 4.
> Il est normal que les réponses ne soient pas très bonnes à ce stade. On vous demande d’expliquer **pourquoi**.